# Twitter Trading Strategy
Letting Tweet Sentiment determine buy, sell, or holds.

In [1]:
# Libraries
import twint
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import plotly.express as px
import yfinance as yf